## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from model import MyModel
from tqdm import tqdm
from albumentations import *
from albumentations.pytorch import ToTensorV2

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

## 1. Model 정의

In [3]:
# class MyModel(nn.Module):
#     def __init__(self, num_classes: int = 1000):
#         super(MyModel, self).__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
#             nn.BatchNorm2d(64),
#             nn.ReLU(inplace=True),
#         )
#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.classifier = nn.Sequential(
#             nn.Dropout(),
#             nn.Linear(64, 32),
#             nn.ReLU(inplace=True),
#             nn.Linear(32, num_classes),
#         )

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         x = self.features(x)
#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.classifier(x)
#         return x

## 2. Test Dataset 정의

In [4]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image=np.array(image))['image']
        return image.float()

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [11]:
model_name = 'efficientnet_b3a_aug_chpolicy_trfm_probup'
models = [
 f'../models/{model_name}/fold_1_{model_name}_best.pt',
   f'../models/{model_name}/fold_2_{model_name}_best.pt',
    f'../models/{model_name}/fold_3_{model_name}_best.pt',
    f'../models/{model_name}/fold_4_{model_name}_best.pt',
    f'../models/{model_name}/fold_5_{model_name}_best.pt'    
]

In [12]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform_valid = Compose([
        CenterCrop(height=384, width=384, p=1.0),
        #RandomCrop(always_apply=True, height=384, width=384, p=1.0),
        Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246), max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
        #T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform_valid)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
results = np.array([0]*18)
all_predictions = []
for model_path in models:
    print(f'Inference {model_path}')
    model = torch.load(model_path).to(device)

    #MyModel(num_classes=18).to(device)

    model.eval()

    # 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
    all_prediction = []
    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            #pred = pred.argmax(dim=-1)
            all_prediction.extend(pred.cpu().numpy())
    all_predictions.append(all_prediction)

print(np.array(all_predictions).shape)
np.save(f'../results/fold_{model_name}_predict_result_center_crop.npy', np.array(all_predictions))
all_predictions = np.array(all_predictions)
master_predictions = all_predictions[0] + all_predictions[1] + all_predictions[2] + all_predictions[3] + all_predictions[4]
ensemble_result = np.argmax(master_predictions, axis=1)
print(ensemble_result.shape)

submission['ans'] = ensemble_result
submission.to_csv(f'../results/submision_fold_{model_name}_center_crop.csv', index=False)
print('test inference is done!')

# # print(results[:5])
# # print(np.argmax(results, dim=1)[:5])
# # submission['ans'] = np.argmax(results, dim=1)

# # #제출할 파일을 저장합니다.
# # submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
# # print('test inference is done!')

  0%|          | 0/12600 [00:00<?, ?it/s]

Inference ../models/efficientnet_b3a_aug_chpolicy_trfm_probup/fold_1_efficientnet_b3a_aug_chpolicy_trfm_probup_best.pt


  0%|          | 0/12600 [00:00<?, ?it/s]

Inference ../models/efficientnet_b3a_aug_chpolicy_trfm_probup/fold_2_efficientnet_b3a_aug_chpolicy_trfm_probup_best.pt


100%|██████████| 12600/12600 [04:38<00:00, 45.18it/s]


...

Inference ../models/efficientnet_b3a_aug_chpolicy_trfm_probup/fold_4_efficientnet_b3a_aug_chpolicy_trfm_probup_best.pt


100%|██████████| 12600/12600 [05:50<00:00, 35.97it/s]


Inference ../models/efficientnet_b3a_aug_chpolicy_trfm_probup/fold_5_efficientnet_b3a_aug_chpolicy_trfm_probup_best.pt


100%|██████████| 12600/12600 [05:46<00:00, 36.42it/s]


(5, 12600, 18)
(12600,)
test inference is done!


In [8]:
print(ensemble_result)

[13  1 13 ...  9  1  7]


In [87]:
print(ensemble_result)

[13  1 13 ...  9  1  7]


In [52]:
# for i in range(5):
#     re = np.unique(np.argmax(all_predictions[i],axis=1), return_counts=True)
# #     print(i)
# #     for j in range(len(re[0])):
# #         print(re[0][j], ":", re[1][j])

In [55]:
idx = 3
submission['ans'] = np.argmax(all_predictions[idx],axis=1)
submission.to_csv(os.path.join(test_dir, f'submission_{idx}.csv'), index=False)

In [ ]:
torch.optim.AdamP

In [ ]:
#[13  1 13 ...  9  1  7]

In [9]:
model = torch.load('../models/efficientnet_b3a_aug_plus_ch_trfm_normalize/fold_1_efficientnet_b3a_aug_plus_ch_trfm_normalize_best.pt').to(device)
#MyModel(num_classes=18).to(device)

model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_prediction = []
for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_prediction.extend(pred.cpu().numpy())

100%|██████████| 12600/12600 [06:14<00:00, 33.61it/s]


In [10]:
submission['ans'] = all_prediction
submission.to_csv(os.path.join(test_dir, './submission_non_ensemble.csv'), index=False)

In [74]:
len(all_prediction)

12600

In [75]:
np.unique(all_prediction, return_counter=True)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17])

In [7]:
submission.to_csv(f'../results/submision_fold_{model_name}_center_crop.csv', index=False)